# Leer y cargar archivos con DiPy y nibabel

De la base de datos de DiPy descargamos una muestra con tres archivos básicos:
* Un archivo en formato Nifti (.nii) con el volumen de imágenes.
* Un archivo con los valores b.
* Un archivo con los vectores gradiente.

In [3]:
from dipy.data import fetch_sherbrooke_3shell
fetch_sherbrooke_3shell() # Download a 3shell HARDI dataset with 192 gradient direction

Creating new folder /home/david/.dipy/sherbrooke_3shell
Download Progress: [##################################] 100.00%  of 183.58 MBDownloading "HARDI193.bval" to /home/david/.dipy/sherbrooke_3shell
Download Progress: [##################################] 100.00%  of 0.00 MBDownloading "HARDI193.bvec" to /home/david/.dipy/sherbrooke_3shell
Download Progress: [##################################] 100.00%  of 0.01 MBFiles successfully downloaded to /home/david/.dipy/sherbrooke_3shell


({'HARDI193.nii.gz': ('https://digital.lib.washington.edu/researchworks/bitstream/handle/1773/38475/HARDI193.nii.gz',
   '0b735e8f16695a37bfbd66aab136eb66'),
  'HARDI193.bval': ('https://digital.lib.washington.edu/researchworks/bitstream/handle/1773/38475/HARDI193.bval',
   'e9b9bb56252503ea49d31fb30a0ac637'),
  'HARDI193.bvec': ('https://digital.lib.washington.edu/researchworks/bitstream/handle/1773/38475/HARDI193.bvec',
   '0c83f7e8b917cd677ad58a078658ebb7')},
 '/home/david/.dipy/sherbrooke_3shell')

In [7]:
from os.path import expanduser, join
home = expanduser('~')
dname = join(home, '.dipy', 'sherbrooke_3shell')

Nombres de los 3 archivos:

In [9]:
fdwi = join(dname, 'HARDI193.nii.gz')
fbval = join(dname, 'HARDI193.bval')
fbvec = join(dname, 'HARDI193.bvec')
print(fdwi)
print(fbval)
print(fbvec)

/home/david/.dipy/sherbrooke_3shell/HARDI193.nii.gz
/home/david/.dipy/sherbrooke_3shell/HARDI193.bval
/home/david/.dipy/sherbrooke_3shell/HARDI193.bvec


Cargamos los archivos con la librería ```nibabel```:

In [14]:
import nibabel as nib
img = nib.load(fdwi)
data = img.get_data() # 4D array
print(data.shape)

(128, 128, 60, 193)


```data``` es un volumen de 60 imágenes de 128x128 con 196 direcciones gradiente.